This code extracts the tweets from a particular user's timeline

In [1]:
import tweepy
from tweepy import OAuthHandler
import pandas as pd

# Consumer/Access key/secret/token obtained from Twitter
# You should have created a Twitter app and gotten these keys.
# Do NOT share your key/secret/token with other students.
consumer_key    = ''
consumer_secret = ''
access_token    = ''
access_secret   = ''

# The following two lines create an authorization object with your above authentication info.
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

# This line finally calls Twitter's Rest API.
api = tweepy.API(auth)

In [8]:
# Main code that extracts and geocode tweets
df = pd.DataFrame(columns=['id','tweet','animal','location', 'lat', 'long'])
latlong = pd.read_excel("SingaporeLatLong2.xlsx")
screen_name = '@ctrlALF'

for status in tweepy.Cursor(api.user_timeline, screen_name=screen_name, tweet_mode="extended").items():
    tweet_id = status.id
    status_text = status.full_text
    animal = "Not detected/Others"
    location = None
    lat = None
    long = None
    
    if "dog" in status_text.lower():
        animal = "Dog"
    if "cat" in status_text.lower():
        animal = "Cat"
    
    if "LOST" in status_text:
        for index, row in latlong.iterrows():
            if row.location.lower() in status_text.lower():
                location = row.location
                lat = row.latitude
                long = row.longitude
                break
    
    df.loc[len(df)] = [tweet_id, status_text, animal, location, lat, long]
    
df

,id,tweet,animal,location,lat,long
0,1233906806109298688,p.s. These furbabies are best adopted by those...,Cat,None,NaN,NaN
1,1233905556051525632,ADOPT: These two FIV+ cats are looking for hom...,Cat,None,NaN,NaN
2,1233399631544930304,LOST: Northlanders! Has any of you seen this c...,Cat,Woodlands,1.436897,103.786216
3,1232354240598659072,LOST: Cat missing since 2 days ago. Last seen ...,Cat,Hougang,1.371904,103.892725
4,1231225206376501248,"LOST: Bishan residents, heads up! Lost cat las...",Cat,Bishan,1.351452,103.848250
...,...,...,...,...,...,...
180,438898795011510272,LOST: Missing cat called Baby. Last seen aroun...,Cat,Bukit Panjang,1.378629,103.762136
181,438139312144531456,"No problem, dear :) RT @stxticflora: @ALFina_S...",Not detected/Others,None,NaN,NaN
182,436489648299200514,"LOST: Tri-colour, female dog called Rina. Last...",Dog,Faber,1.271924,103.817175
183,436488810117861377,FOUND: 8-10 year old female Jack Russell terri...,Not detected/Others,None,NaN,NaN


In [9]:
# Check results for tweets with location
for index, row in df.iterrows():
    if row.location:
        print("Animal: " + row.animal)
        print("Location: " + row.location, row.lat, row.long)
        if row.animal == "Not detected/Others":
            print(row.tweet)
        print(" ")

Animal: Cat
Location: Woodlands 1.4368969999999999 103.78621600000001
 
Animal: Cat
Location: Hougang 1.3719043 103.892725
 
Animal: Cat
Location: Bishan 1.3514521000000002 103.84824959999999
 
Animal: Cat
Location: Bukit Panjang 1.378629 103.7621358
 
Animal: Not detected/Others
Location: Bukit Panjang 1.378629 103.7621358
LOST: Black, male Scottish Fold last seen at Blk 227 Pending Rd, Bukit Panjang. If spotted, do text the number below or dm via twitter id. Do spread the word! (ctt: @haylda)-31/1 https://t.co/mTKVsBKBbL
 
Animal: Cat
Location: Jurong West 1.3396365 103.7073387
 
Animal: Not detected/Others
Location: Yishun 1.4293839 103.8350282
LOST: Smokey, the persian, was last seen this morning at Yishun Blk 459. Afraid of people but will respond to her name. If found, please contact owner at 88177282 or via twitter. Northlanders, do your part &amp; RT! (ctt: @M4I1R4)-7/1 https://t.co/o8BuBvrMDX
 
Animal: Cat
Location: Taman Jurong 1.33488355 103.72078867428404
 
Animal: Cat
Loca

In [10]:
# Export results to xlsx
df.to_excel('{}\'s_tweets.xlsx'.format(screen_name))